In [ ]:
import pandas as pd
from prophet import Prophet
import matplotlib.pyplot as plt

def preprocess(data, split_date, column_mapping):
    data.rename(columns=column_mapping, inplace=True)
    data['ds'] = pd.to_datetime(data['ds'])
    data['ds'] = data['ds'].dt.tz_localize(None)
    #print(data.head())

    train_data = data[data['ds'] < split_date]  # Дані до вказаної дати
    actual_data = data[data['ds'] >= split_date]   # Дані після вказаної дати
    return train_data, actual_data

def get_holidays():
    holidays = pd.DataFrame({
        'holiday': 'blackfriday',
        'ds': pd.to_datetime(['2020-11-27', '2021-11-26', '2022-11-25',
                        '2023-11-24', '2024-11-29']),
        'lower_window': -4,
        'upper_window': 3,
    })
    return holidays

def predict(
    train_data,
    periods,
    holidays=None,
    seasonality_mode='additive',
    interval_width=0.8,
):
    model = Prophet(
        holidays=holidays,
        seasonality_mode=seasonality_mode,
        interval_width=interval_width
    )
    model.fit(train_data)

    future = model.make_future_dataframe(periods, include_history=True)
    forecast = model.predict(future)
    print('future=', future)
    model.plot_components(forecast)
    model.plot(forecast)
    return forecast

def plot(train_data, forecast, comparison_df, yhat, periods):
    # Візуалізація прогнозу та фактичних даних
    plt.figure(figsize=(10, 6))
    plt.plot(train_data['ds'].head(periods), train_data['y'].head(periods), label='Тренувальні дані')
    plt.plot(forecast['ds'].tail(periods), forecast[yhat].tail(periods), label='Прогноз', linestyle='--')
    plt.plot(comparison_df['ds'], comparison_df['y'], label='Фактичні дані', linestyle=':', marker='o')
    plt.fill_between(forecast['ds'].tail(periods), forecast['yhat_lower'].tail(periods), forecast['yhat_upper'].tail(periods), color='gray', alpha=0.2)
    plt.legend()
    plt.xlabel('Дата')
    plt.ylabel('Цільова змінна')
    plt.title('Прогноз Prophet та фактичні дані')
    plt.grid(True)
    plt.show()

In [ ]:
import yfinance as yf
from datetime import datetime

ticker = 'SPY'
split_date = '2024-11-30'
predict_periods = 45
start = '2020-11-12' # 2018 2019 2021
end = datetime.now().strftime('%Y-%m-%d')

file_data = yf.download(ticker, start=start, end=end)
data_reset = file_data.reset_index()

data = pd.DataFrame(columns=['Date', 'Close'])
data['Date'] = data_reset['Date']
data['Close'] = data_reset['Close']
data = data[::-1].reset_index(drop=True)

# завантажено з https://www.nasdaq.com/market-activity/etf/spy/historical?page=1&rows_per_page=10&timeline=y10
#file_path = 'files/spy_till_2024-11-11.csv'
#data = pd.read_csv(file_path, usecols=['Date', 'Close/Last'])

print(data)
column_mapping = {'Date': 'ds', 'Close': 'y' }
train_data, actual_data = preprocess(data, split_date, column_mapping)
forecast = predict(train_data, predict_periods)

yhat = 'yhat'
comparison_df = forecast[['ds', yhat]].merge(actual_data, on='ds', how='left')
#print(comparison_df.tail(20))

plot(train_data, forecast, comparison_df, yhat, predict_periods)